In [1]:
from pyspark.sql import SparkSession, functions as f, types as t

In [2]:
spark = SparkSession.builder.master('local[3]').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/14 09:29:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(spark.version)

3.3.1


In [4]:
actor_df = spark.read.csv('./data/actor.csv', header=True, inferSchema=True)
address_df = spark.read.csv('./data/address.csv', header=True, inferSchema=True)
category_df = spark.read.csv('./data/category.csv', header=True, inferSchema=True)
city_df = spark.read.csv('./data/city.csv', header=True, inferSchema=True)
country_df = spark.read.csv('./data/country.csv', header=True, inferSchema=True)
customer_df = spark.read.csv('./data/customer.csv', header=True, inferSchema=True)
film_df = spark.read.csv('./data/film.csv', header=True, inferSchema=True)
film_actor_df = spark.read.csv('./data/film_actor.csv', header=True, inferSchema=True)
film_category_df = spark.read.csv('./data/film_category.csv', header=True, inferSchema=True)
inventory_df = spark.read.csv('./data/inventory.csv', header=True, inferSchema=True)
language_df = spark.read.csv('./data/language.csv', header=True, inferSchema=True)
payment_df = spark.read.csv('./data/payment.csv', header=True, inferSchema=True)
rental_df = spark.read.csv('./data/rental.csv', header=True, inferSchema=True)
staff_df = spark.read.csv('./data/staff.csv', header=True, inferSchema=True)
store_df = spark.read.csv('./data/store.csv', header=True, inferSchema=True)

# Домашнє завдання на тему Spark SQL

Задачі з домашнього завдання на SQL потрібно розвʼязати за допомогою Spark SQL DataFrame API.

- Дампи таблиць знаходяться в папці `data`. Датафрейми таблиць вже створені в клітинці вище.
- Можете створювати стільки нових клітинок, скільки вам необхідно.
- Розвʼязок кожної задачі має бути відображений в самому файлі (використати метод `.show()`)
- код має бути оформлений у відповідності із одним із стилем, показаним лектором на занятті 13.

**Увага!**
Використовувати мову запитів SQL безпосередньо забороняється, потрібно використовувати виключно DataFrame API!


1.
Вивести кількість фільмів в кожній категорії.
Результат відсортувати за спаданням.

In [5]:
category_df.join(
    film_category_df, on="category_id", how='inner'
).groupby(
    f.col('name').alias('category')
).agg(
    f.count('film_id').alias('films')
).orderBy(
    f.desc('films')
).show()

+-----------+-----+
|   category|films|
+-----------+-----+
|     Sports|   74|
|    Foreign|   73|
|     Family|   69|
|Documentary|   68|
|  Animation|   66|
|     Action|   64|
|        New|   63|
|      Drama|   62|
|      Games|   61|
|     Sci-Fi|   61|
|   Children|   60|
|     Comedy|   58|
|     Travel|   57|
|   Classics|   57|
|     Horror|   56|
|      Music|   51|
+-----------+-----+



2.
Вивести 10 акторів, чиї фільми брали на прокат найбільше.
Результат відсортувати за спаданням.

In [6]:
actor_df.join(
    film_actor_df, on='actor_id', how='inner'
).join(
    inventory_df, on='film_id', how='inner'
).join(
    rental_df, on='inventory_id', how='inner'
).groupBy(
    'actor_id', f.concat('first_name', f.lit(' '), 'last_name').alias('actor')
).agg(
    f.count('inventory_id').alias('rentals')
).orderBy(
    f.desc('rentals')
).select(
    'actor'
).show(10)

+------------------+
|             actor|
+------------------+
|    GINA DEGENERES|
|    MATTHEW CARREY|
|       MARY KEITEL|
|ANGELA WITHERSPOON|
|       WALTER TORN|
|       HENRY BERRY|
|       JAYNE NOLTE|
|        VAL BOLGER|
|     SANDRA KILMER|
|      SEAN GUINESS|
+------------------+
only showing top 10 rows



3.
Вивести категорія фільмів, на яку було витрачено найбільше грошей
в прокаті

In [7]:
category_df.join(
    film_category_df, on='category_id', how='inner'
).join(
    inventory_df, on='film_id', how='inner'
).join(
    rental_df, on='inventory_id', how='inner'
).join(
    payment_df, on='rental_id', how='inner'
).groupBy(
    'category_id', 'name'
).agg(
    f.sum('amount').alias('total_amount')
).orderBy(
    f.desc('total_amount')
).select(
    'name'
).show(1)

+------+
|  name|
+------+
|Sports|
+------+
only showing top 1 row



4.
Вивести назви фільмів, яких не має в inventory.

In [11]:
film_df.join(
    inventory_df, on='film_id', how='left'
).select(
    'title'
).where(
    'inventory_id is null'
).orderBy(
    'title'
).show()

+--------------------+
|               title|
+--------------------+
|      ALICE FANTASIA|
|         APOLLO TEEN|
|      ARGONAUTS TOWN|
|       ARK RIDGEMONT|
|ARSENIC INDEPENDENCE|
|   BOONDOCK BALLROOM|
|       BUTCH PANTHER|
|       CATCH AMISTAD|
| CHINATOWN GLADIATOR|
|      CHOCOLATE DUCK|
|COMMANDMENTS EXPRESS|
|    CROSSING DIVORCE|
|     CROWDS TELEMARK|
|    CRYSTAL BREAKING|
|          DAZED PUNK|
|DELIVERANCE MULHO...|
|   FIREHOUSE VIETNAM|
|       FLOATS GARDEN|
|FRANKENSTEIN STRA...|
|  GLADIATOR WESTWARD|
+--------------------+
only showing top 20 rows



5.
Вивести топ 3 актори, які найбільше зʼявлялись в категорії фільмів “Children”

In [22]:
actor_df.join(
    film_actor_df, on='actor_id'
).join(
    film_category_df, on='film_id'
).join(
    category_df, on='category_id'
).where(
    category_df['name'] == 'Children'
).withColumn(
    'actor',
    f.concat('first_name', f.lit(' '), 'last_name')
).groupBy(
    'actor_id', 'actor'
).agg(
    f.count(film_actor_df['film_id']).alias('films')
).orderBy(
    f.desc('films'), 'actor'
).limit(
    3
).select(
    'actor'
).show()

+-------------+
|        actor|
+-------------+
| HELEN VOIGHT|
|KEVIN GARLAND|
|   MARY TANDY|
+-------------+



Stop Spark session:

In [23]:
spark.stop()